# Universal Company Brochure Generator

Generate company brochures from a URL using various LLMs (OpenAI, Anthropic, Google, Groq) and tonalities.

In [1]:
# Imports
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Load Environment Variables
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

print("API Keys Loaded")

API Keys Loaded


In [2]:
# Initialize Clients

openai_client = OpenAI(api_key=openai_api_key)

anthropic_url = "https://api.anthropic.com/v1/"
anthropic_client = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_client = OpenAI(api_key=google_api_key, base_url=gemini_url)

groq_url = "https://api.groq.com/openai/v1"
groq_client = OpenAI(api_key=groq_api_key, base_url=groq_url)

In [3]:
# Configuration

MODELS = {
    "GPT-4o": {"client": openai_client, "model": "gpt-4o"},
    "GPT-4o Mini": {"client": openai_client, "model": "gpt-4o-mini"},
    "Claude 3.5 Sonnet": {"client": anthropic_client, "model": "claude-3-5-sonnet-20241022"},
    "Claude 3 Haiku": {"client": anthropic_client, "model": "claude-3-haiku-20240307"},
    "Gemini 1.5 Pro": {"client": google_client, "model": "gemini-1.5-pro"},
    "Gemini 1.5 Flash": {"client": google_client, "model": "gemini-1.5-flash"},
    "Llama 3.3 70B (Groq)": {"client": groq_client, "model": "llama-3.3-70b-versatile"},
    "Llama 3.1 8B (Groq)": {"client": groq_client, "model": "llama-3.1-8b-instant"}
}

TONALITIES = {
    "Professional": "Write in a strictly professional, corporate tone suitable for B2B clients.",
    "Casual": "Write in a friendly, relaxed, and conversational tone, like you're talking to a friend.",
    "Humorous": "Write in a funny, witty style with some jokes and lighthearted banter.",
    "Sarcastic": "Write in a cynical, slightly mocking tone, pointing out the absurdities of corporate speak.",
    "Enthusiastic": "Write with high energy, using exclamation marks and hype! Everything is amazing!"
}

In [4]:
# Website Scraper function

def fetch_website_contents(url):
    print(f"Fetching {url}...")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
            
        text = soup.get_text()
        
        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        
        # Truncate to avoid context window issues (rough estimate)
        return text[:10000]
    except Exception as e:
        return f"Error fetching website: {str(e)}"

In [5]:
# Brochure Generator Function

def stream_brochure(company_name, url, model_name, tonality):
    yield "Fetching website contents..."
    
    website_content = fetch_website_contents(url)
    
    if website_content.startswith("Error"):
        yield website_content
        return

    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n\n{website_content}"
    
    tonality_instruction = TONALITIES.get(tonality, "Write in a professional tone.")
    system_message = f"You are an assistant that analyzes the contents of a company website landing page and creates a short brochure. {tonality_instruction} Respond in markdown without code blocks."
    
    # Get client and model ID
    config = MODELS[model_name]
    client = config["client"]
    model_id = config["model"]
    
    yield f"Generating brochure using {model_name}..."
    
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
        
        stream = client.chat.completions.create(
            model=model_id,
            messages=messages,
            stream=True
        )
        
        result = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                yield result
                
    except Exception as e:
        yield f"Error generating brochure: {str(e)}"

In [6]:
# Gradio Interface

company_input = gr.Textbox(label="Company Name", placeholder="e.g., Anthropic")
url_input = gr.Textbox(label="Website URL", placeholder="e.g., https://anthropic.com")
model_dropdown = gr.Dropdown(choices=list(MODELS.keys()), label="Select Model", value="GPT-4o Mini")
tonality_dropdown = gr.Dropdown(choices=list(TONALITIES.keys()), label="Select Tonality", value="Professional")
output_markdown = gr.Markdown(label="Generated Brochure")

demo = gr.Interface(
    fn=stream_brochure,
    inputs=[company_input, url_input, model_dropdown, tonality_dropdown],
    outputs=[output_markdown],
    title="Universal Company Brochure Generator",
    description="Generate a custom company brochure using any LLM model with a specific tonality.",
    examples=[
        ["Hugging Face", "https://huggingface.co", "GPT-4o Mini", "Enthusiastic"],
        ["OpenAI", "https://openai.com", "Claude 3.5 Sonnet", "Sarcastic"]
    ],
    flagging_mode="never"
)

if __name__ == '__main__':
    demo.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
